In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.utils.data import Subset
import numpy as np

import pandas as pd
import os
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import random


In [2]:
kaggle_env = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', 'Localhost') == 'Interactive'
kaggle_env

True

In [3]:
if not kaggle_env:
    dottless_text_file = 'dotless_text.txt'
    original_text_file = 'original_text.txt'
else: 
    dottless_text_file = '/kaggle/input/arabic-dottization-dataset/dotless_text.txt'
    original_text_file = '/kaggle/input/arabic-dottization-dataset/original_text.txt'

# load the data
with open(dottless_text_file, 'r', encoding='utf-8') as f:
    dottless_text_list = f.readlines()

with open(original_text_file, 'r', encoding='utf-8') as f:
    original_text_list = f.readlines()


print(dottless_text_list[3])
print(original_text_list[3])

 ٮوحد ٮوعاں مں محطاٮ الٮلڡار: المحطاٮ الٮحارٮه والمحطاٮ العامه. ٮدار المحطاٮ الٮحارٮه ٮواسطه سركاٮ حاصه. وٮٮٮع هده السركاٮ وڡٮ الإعلاٮاٮ لٮعطٮه ٮڡڡاٮ الٮسعٮل، ٮالإصاڡه لٮحڡٮٯ رٮح للسركاٮ الٮى ٮدٮر المحطاٮ. أما محطاٮ الٮلڡار العامه ڡهى محطاٮ لا ٮهدڡ إلى الرٮح وٮدار وڡٯ ٮرٮٮٮاٮ حاصه. ڡمٮلا ٮحصل هٮئه الإداعه الٮرٮطاٮٮه على الٮموٮل مں رسوم الٮرحٮص الٮى ٮدڡعها مالكو أحهره الٮلڡار وهى لا ٮٮٮع وڡٮا للإعلاٮاٮ. وٮعٮمد محطاٮ الٮلڡار العامه ڡى معطم الدول، على مساهماٮ ڡطاع الأعمال، والحكومه، والحمهور، ودلك لٮعطٮه ٮڡڡاٮ الٮسعٮل، ومں ٮم ڡإٮهم ٮٮحدوں ڡراراٮهم ٮسأں محٮوٮاٮ الٮرامح ٮأٮڡسهم. وڡى دول أحرى ٮڡوم الحكوماٮ ٮإداره محطاٮ الٮلڡار الٮى ٮٮحد الڡراراٮ ٮسأں محٮوٮاٮ الٮرامح. وٮصڡه عامه لا ٮٮٮع هده المحطاٮ وڡٮا للإعلاں. ٮسٮطٮع الأڡراد ڡى ٮعص الدول الاسٮراك ڡى أٮطمه الٮلڡار الكٮلى (حط الماٮكرووٮڡ) وأٮطمه الٮٮ لمساهدوں رسوما لهده الحدماٮ. كما ٮوحد اسٮحداماٮ أحرى للٮلڡار عٮر ٮٮ الٮرامح للمٮارل. ڡمٮلا، ٮسٮحدم المدارس، وڡطاع الأعمال، والمسٮسڡٮاٮ، وعٮرها مں المٮطماٮ دوائر الٮلڡار المعلڡه. وٮرسل الإساراٮ ڡى

In [4]:
def get_vocab(data):
    vocab = set()
    for sentence in data:
        for letter in sentence:
            vocab.update(letter)
            
    vocab = sorted(vocab)
    
    char2index = {'<PAD>': 0}
    index2char = {0: '<PAD>'}
    for index, char in enumerate(vocab, len(index2char)):
        char2index[char] = index
        index2char[index] = char

    return vocab, char2index, index2char

# Create a vocab
dottless_chars, dottless_char2index, dottless_index2char = get_vocab(dottless_text_list)
original_chars, original_char2index, original_index2char = get_vocab(original_text_list)

# Convert to index
dottless_text_list_encoded = []
for sentence in dottless_text_list: 
    dottless_text_list_encoded.append([dottless_char2index[char] for char in sentence])

original_text_list_encoded = []
for sentence in original_text_list:
    original_text_list_encoded.append([original_char2index[char] for char in sentence])

In [5]:
def pad_sequence(x, max_len, pad_token_index=0):
    padded = np.full((max_len), fill_value=pad_token_index)
    if len(x) > max_len: padded[:] = x[:max_len]
    else: padded[:len(x)] = x
    return padded

# pad the sequences
max_length = 512
dottless_text_list_padded = [pad_sequence(sequence, max_length) for sequence in dottless_text_list_encoded]
original_text_list_padded = [pad_sequence(sequence, max_length) for sequence in original_text_list_encoded]

In [6]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(dottless_text_list_padded, original_text_list_padded, test_size=0.1, random_state=42)
 

In [7]:
class ArabicDottizationDataset(Dataset):
    def __init__(self, dottless_text, original_text):
        self.dottless_text = dottless_text
        self.original_text = original_text
         
    def __len__(self):
        return len(self.dottless_text)

    def __getitem__(self, index):     
        return torch.tensor(self.dottless_text[index]), torch.tensor(self.original_text[index])


In [8]:
train_dataset = ArabicDottizationDataset(x_train, y_train)
test_dataset = ArabicDottizationDataset(x_test, y_test)
  

# The Model

In [9]:
class ArabicDottizationModel(nn.Module):
    def __init__(self, dotless_vocab_size, dotted_vocab_size, embedding_dim, hidden_size):
        super().__init__()
        
        self.embedding = nn.Embedding(dotless_vocab_size, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_size, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, dotted_vocab_size)
        
        
    def forward(self, dotless_sentence):
        embeddings = self.embedding(dotless_sentence)  
        embeddings = embeddings.transpose(0, 1)  
        output, _ = self.rnn(embeddings)  
        output = self.fc(output)  
        output = output.transpose(0, 1)  
        return output


In [10]:
embed_size = 512
hidden_size = 256
dottless_vocab_size = len(dottless_char2index)
dotted_vocab_size = len(original_char2index)
learning_rate = 1e-4
num_epochs = 20
batch_size = 128

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = ArabicDottizationModel(dottless_vocab_size, dotted_vocab_size, embed_size, hidden_size).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=original_char2index['<PAD>'])
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [12]:
model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in tqdm(
        enumerate(train_loader), total=len(train_loader), leave=False
    ):
        inputs, labels = inputs.to(device), labels.to(device).long()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
         
        outputs = outputs.reshape(-1, dotted_vocab_size) 
        labels = labels.reshape(-1)
         
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print('Loss: {}'.format(running_loss))

print('Finished Training')

  0%|          | 0/79 [00:00<?, ?it/s]

Loss: 180.86547231674194


  0%|          | 0/79 [00:00<?, ?it/s]

Loss: 46.453004002571106


  0%|          | 0/79 [00:00<?, ?it/s]

...

Loss: 15.40352013707161


  0%|          | 0/79 [00:00<?, ?it/s]

Loss: 14.985851496458054


  0%|          | 0/79 [00:00<?, ?it/s]

Loss: 14.604834854602814
Finished Training


In [21]:
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


In [42]:
 
model.eval()

for image, gt in test_loader:
    image = image.to(device)
    outputs = model(image) 
    predicted = outputs.argmax(2).cpu() 
    print('Dottless Text:')
    print(''.join([dottless_index2char[c.item()] for c in image[0]]))

    print('\nModel Text:') 
    print(''.join([original_index2char[c.item()] for c in predicted[0]]))
 
    print('\nActual Text:')
    print(''.join([original_index2char[c.item()] for c in gt[0]])) 
    print('-' * 40)
    

Dottless Text:
أٮار ڡرٮه مں مارعرٮٮ العٮره ڡى والدٮه، الٮى حاولٮ أں ٮڡصل ٮٮں الروحٮں ڡدر اسٮطاعٮها. حمله الڡدٮس لوٮس الصلٮٮٮه  ڡى عام 1229، عٮدما كاں لوٮس ٮٮلع مں العمر 15 عاما، أٮهٮ والدٮه الحمله الصلٮٮٮه الألٮٮحٮٮٮه ٮٮوڡٮع اٮڡاڡٮه مع رٮموٮد الساٮع ملك ٮولور. كاں رٮموں السادس ملك ٮولور ٮسٮٮه ڡى أٮه أمر ٮاعٮٮال ٮٮٮر دى كاسٮٮلٮاو، وهو واعط كاٮولٮكى روماٮى حاول ٮحوٮل الكاٮارٮٮں. ڡاد لوٮس حملٮٮں صلٮٮٮٮٮں: الحمله الصلٮٮٮه الساٮعه عام 1248 والحمله الصلٮٮٮه الٮامٮه عام 1270. الحمله الصلٮٮٮه الساٮعه  وصل لوٮس وأٮٮاعه إلى مصر ڡى 

Model Text:
أبار قرية من مارغريت الغيرة في والدية، التي حاولت أن يفصل بين الروحين قدر استطاعتها. حملة القديس لويس الصليبية  في عام 1229، عندما كان لويس ببلغ من العمر 15 عاما، أنهب والدية الحملة الصليبية الألييجييية تتوفيع اتقافية مع ريمويد السابع ملك بولور. كان ريمون السادس ملك بولوز بسبية في أنه أمر باعتبال ببير ذي كاستيلياو، وهو واعط كاتوليكي روماني حاول تحويل الكاباريين. فاد لويس حمليين صلييييين: الحملة الصليبية السابعة عام 1248 والحملة الصليبية التامية عام 1270. 